# Using ipython-sql

In [3]:
%load_ext sql

In [4]:
import os

host = "localhost"
database = "dataset"
user = os.getenv('SQL_USER')
password = os.getenv('SQL_PASSWORD')

connection_string = f"postgresql://{user}:{password}@{host}/{database}"

In [5]:
%sql $connection_string

'Connected: postgres@dataset'

# Using sqlalchemy

In [6]:
from sqlalchemy import create_engine, inspect

engine = create_engine(connection_string)
insp = inspect(engine)
insp.get_table_names()

['startups',
 'ads_ctr_optimisation',
 'churn_modelling',
 'data',
 'mall_customers',
 'position_salaries',
 'salary_data',
 'social_network_ads',
 'wine']

# Import data

In [7]:
import pandas as pd

df = pd.read_sql('SELECT * FROM churn_modelling', engine)

In [8]:
df

,rownumber,customerid,surname,creditscore,geography,gender,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,exited
0,1,15634602.0,Hargrave,619.0,France,Female,42.0,2.0,0.00,1.0,1,1,101348.88,1
1,2,15647311.0,Hill,608.0,Spain,Female,41.0,1.0,83807.86,1.0,0,1,112542.58,0
2,3,15619304.0,Onio,502.0,France,Female,42.0,8.0,159660.80,3.0,1,0,113931.57,1
3,4,15701354.0,Boni,699.0,France,Female,39.0,1.0,0.00,2.0,0,0,93826.63,0
4,5,15737888.0,Mitchell,850.0,Spain,Female,43.0,2.0,125510.82,1.0,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229.0,Obijiaku,771.0,France,Male,39.0,5.0,0.00,2.0,1,0,96270.64,0
9996,9997,15569892.0,Johnstone,516.0,France,Male,35.0,10.0,57369.61,1.0,1,1,101699.77,0
9997,9998,15584532.0,Liu,709.0,France,Female,36.0,7.0,0.00,1.0,0,1,42085.58,1
9998,9999,15682355.0,Sabbatini,772.0,Germany,Male,42.0,3.0,75075.31,2.0,1,0,92888.52,1


In [11]:
import numpy as np
import tensorflow as tf
from sklearn.impute import SimpleImputer

dataset = df[['rownumber', 'customerid', 'surname', 'gender', 'geography',
                   'creditscore', 'age', 'tenure', 'balance', 'numofproducts', 'hascrcard',
                   'isactivemember', 'estimatedsalary', 'exited']]
dataset

,rownumber,customerid,surname,gender,geography,creditscore,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,exited
0,1,15634602.0,Hargrave,Female,France,619.0,42.0,2.0,0.00,1.0,1,1,101348.88,1
1,2,15647311.0,Hill,Female,Spain,608.0,41.0,1.0,83807.86,1.0,0,1,112542.58,0
2,3,15619304.0,Onio,Female,France,502.0,42.0,8.0,159660.80,3.0,1,0,113931.57,1
3,4,15701354.0,Boni,Female,France,699.0,39.0,1.0,0.00,2.0,0,0,93826.63,0
4,5,15737888.0,Mitchell,Female,Spain,850.0,43.0,2.0,125510.82,1.0,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229.0,Obijiaku,Male,France,771.0,39.0,5.0,0.00,2.0,1,0,96270.64,0
9996,9997,15569892.0,Johnstone,Male,France,516.0,35.0,10.0,57369.61,1.0,1,1,101699.77,0
9997,9998,15584532.0,Liu,Female,France,709.0,36.0,7.0,0.00,1.0,0,1,42085.58,1
9998,9999,15682355.0,Sabbatini,Male,Germany,772.0,42.0,3.0,75075.31,2.0,1,0,92888.52,1


In [12]:
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(X[:, 2:])
X[:, 2:] = imputer.transform(X[:, 2:])

In [13]:
X

array([['Female', 'France', 619.0, ..., 1.0, 1.0, 101348.88],
       ['Female', 'Spain', 608.0, ..., 0.0, 1.0, 112542.58],
       ['Female', 'France', 502.0, ..., 1.0, 0.0, 113931.57],
       ...,
       ['Female', 'France', 709.0, ..., 0.0, 1.0, 42085.58],
       ['Male', 'Germany', 772.0, ..., 1.0, 0.0, 92888.52],
       ['Female', 'France', 792.0, ..., 1.0, 0.0, 38190.78]], dtype=object)

# Model

In [14]:
import tensorflow as tf

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=10, activation='relu'))
model.add(tf.keras.layers.Dense(units=6, activation='relu'))
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Train

In [15]:
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

le = LabelEncoder()
X[:, 0] = le.fit_transform(X[:, 0])
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.fit(X_train, y_train, batch_size = 512, epochs = 1000)

Epoch 1/1000
16/16 [==============================] - 2s 1ms/step - loss: 0.7315 - accuracy: 0.4967
Epoch 2/1000
16/16 [==============================] - 0s 935us/step - loss: 0.6699 - accuracy: 0.6006
Epoch 3/1000
16/16 [==============================] - 0s 951us/step - loss: 0.6241 - accuracy: 0.6733
Epoch 4/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.5895 - accuracy: 0.7241
Epoch 5/1000
16/16 [==============================] - 0s 979us/step - loss: 0.5618 - accuracy: 0.7548
Epoch 6/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.5398 - accuracy: 0.7724
Epoch 7/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.5222 - accuracy: 0.7810
Epoch 8/1000
16/16 [==============================] - 0s 987us/step - loss: 0.5080 - accuracy: 0.7883
Epoch 9/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.4963 - accuracy: 0.7919
Epoch 10/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.4858 - accuracy

16/16 [==============================] - 0s 926us/step - loss: 0.3425 - accuracy: 0.8583
Epoch 82/1000
16/16 [==============================] - 0s 914us/step - loss: 0.3422 - accuracy: 0.8581
Epoch 83/1000
16/16 [==============================] - 0s 930us/step - loss: 0.3417 - accuracy: 0.8599
Epoch 84/1000
16/16 [==============================] - 0s 889us/step - loss: 0.3416 - accuracy: 0.8591
Epoch 85/1000
16/16 [==============================] - 0s 972us/step - loss: 0.3412 - accuracy: 0.8596
Epoch 86/1000
16/16 [==============================] - 0s 959us/step - loss: 0.3408 - accuracy: 0.8594
Epoch 87/1000
16/16 [==============================] - 0s 969us/step - loss: 0.3404 - accuracy: 0.8594
Epoch 88/1000
16/16 [==============================] - 0s 982us/step - loss: 0.3402 - accuracy: 0.8596
Epoch 89/1000
16/16 [==============================] - 0s 1000us/step - loss: 0.3398 - accuracy: 0.8596
Epoch 90/1000
16/16 [==============================] - 0s 983us/step - loss: 0.3396 - 

16/16 [==============================] - 0s 950us/step - loss: 0.3318 - accuracy: 0.8634
Epoch 161/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3317 - accuracy: 0.8630
Epoch 162/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3316 - accuracy: 0.8636
Epoch 163/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3316 - accuracy: 0.8621
Epoch 164/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3315 - accuracy: 0.8634
Epoch 165/1000
16/16 [==============================] - 0s 956us/step - loss: 0.3315 - accuracy: 0.8627
Epoch 166/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3312 - accuracy: 0.8629
Epoch 167/1000
16/16 [==============================] - 0s 968us/step - loss: 0.3313 - accuracy: 0.8630
Epoch 168/1000
16/16 [==============================] - 0s 966us/step - loss: 0.3312 - accuracy: 0.8626
Epoch 169/1000
16/16 [==============================] - 0s 972us/step - loss: 0.3312 - ac

16/16 [==============================] - 0s 983us/step - loss: 0.3283 - accuracy: 0.8635
Epoch 240/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3283 - accuracy: 0.8640
Epoch 241/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3284 - accuracy: 0.8637
Epoch 242/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3285 - accuracy: 0.8645
Epoch 243/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3284 - accuracy: 0.8643
Epoch 244/1000
16/16 [==============================] - 0s 983us/step - loss: 0.3282 - accuracy: 0.8645
Epoch 245/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3282 - accuracy: 0.8641
Epoch 246/1000
16/16 [==============================] - 0s 959us/step - loss: 0.3282 - accuracy: 0.8635
Epoch 247/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.3282 - accuracy: 0.8640
Epoch 248/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3282 - accura

16/16 [==============================] - 0s 895us/step - loss: 0.3271 - accuracy: 0.8645
Epoch 319/1000
16/16 [==============================] - 0s 903us/step - loss: 0.3270 - accuracy: 0.8651
Epoch 320/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3271 - accuracy: 0.8650
Epoch 321/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3272 - accuracy: 0.8639
Epoch 322/1000
16/16 [==============================] - 0s 953us/step - loss: 0.3271 - accuracy: 0.8655
Epoch 323/1000
16/16 [==============================] - 0s 986us/step - loss: 0.3271 - accuracy: 0.8651
Epoch 324/1000
16/16 [==============================] - 0s 915us/step - loss: 0.3270 - accuracy: 0.8655
Epoch 325/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3271 - accuracy: 0.8634
Epoch 326/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3269 - accuracy: 0.8658
Epoch 327/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3269 - ac

...

16/16 [==============================] - 0s 1ms/step - loss: 0.3244 - accuracy: 0.8636
Epoch 635/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3242 - accuracy: 0.8640
Epoch 636/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3239 - accuracy: 0.8645
Epoch 637/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3240 - accuracy: 0.8640
Epoch 638/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3239 - accuracy: 0.8639
Epoch 639/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3240 - accuracy: 0.8637
Epoch 640/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3240 - accuracy: 0.8635
Epoch 641/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3240 - accuracy: 0.8639
Epoch 642/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3240 - accuracy: 0.8639
Epoch 643/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3240 - accuracy: 0.

16/16 [==============================] - 0s 1ms/step - loss: 0.3233 - accuracy: 0.8645
Epoch 715/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3234 - accuracy: 0.8644
Epoch 716/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3234 - accuracy: 0.8646
Epoch 717/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3235 - accuracy: 0.8637
Epoch 718/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3235 - accuracy: 0.8644
Epoch 719/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3235 - accuracy: 0.8648
Epoch 720/1000
16/16 [==============================] - 0s 999us/step - loss: 0.3235 - accuracy: 0.8645
Epoch 721/1000
16/16 [==============================] - 0s 933us/step - loss: 0.3234 - accuracy: 0.8649
Epoch 722/1000
16/16 [==============================] - 0s 996us/step - loss: 0.3235 - accuracy: 0.8639
Epoch 723/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3234 - accura

16/16 [==============================] - 0s 878us/step - loss: 0.3235 - accuracy: 0.8637
Epoch 794/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3234 - accuracy: 0.8640
Epoch 795/1000
16/16 [==============================] - 0s 982us/step - loss: 0.3230 - accuracy: 0.8654
Epoch 796/1000
16/16 [==============================] - 0s 957us/step - loss: 0.3235 - accuracy: 0.8646
Epoch 797/1000
16/16 [==============================] - 0s 959us/step - loss: 0.3232 - accuracy: 0.8655
Epoch 798/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3235 - accuracy: 0.8646
Epoch 799/1000
16/16 [==============================] - 0s 898us/step - loss: 0.3232 - accuracy: 0.8644
Epoch 800/1000
16/16 [==============================] - 0s 958us/step - loss: 0.3232 - accuracy: 0.8640
Epoch 801/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3231 - accuracy: 0.8636
Epoch 802/1000
16/16 [==============================] - 0s 937us/step - loss: 0.3230 

16/16 [==============================] - 0s 998us/step - loss: 0.3231 - accuracy: 0.8640
Epoch 873/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3229 - accuracy: 0.8652
Epoch 874/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3229 - accuracy: 0.8640
Epoch 875/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3229 - accuracy: 0.8649
Epoch 876/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3231 - accuracy: 0.8650
Epoch 877/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3229 - accuracy: 0.8645
Epoch 878/1000
16/16 [==============================] - 0s 960us/step - loss: 0.3230 - accuracy: 0.8648
Epoch 879/1000
16/16 [==============================] - 0s 925us/step - loss: 0.3230 - accuracy: 0.8648
Epoch 880/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3230 - accuracy: 0.8648
Epoch 881/1000
16/16 [==============================] - 0s 917us/step - loss: 0.3230 - accu

16/16 [==============================] - 0s 1ms/step - loss: 0.3229 - accuracy: 0.8641
Epoch 952/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3225 - accuracy: 0.8652
Epoch 953/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3226 - accuracy: 0.8654
Epoch 954/1000
16/16 [==============================] - 0s 990us/step - loss: 0.3227 - accuracy: 0.8644
Epoch 955/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3229 - accuracy: 0.8662
Epoch 956/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3229 - accuracy: 0.8645
Epoch 957/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3226 - accuracy: 0.8661
Epoch 958/1000
16/16 [==============================] - 0s 1ms/step - loss: 0.3226 - accuracy: 0.8651
Epoch 959/1000
16/16 [==============================] - 0s 935us/step - loss: 0.3225 - accuracy: 0.8654
Epoch 960/1000
16/16 [==============================] - 0s 885us/step - loss: 0.3230 - accura

# Evaluate

In [16]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)
print(f"Accuracy Score: {accuracy_score(y_test, y_pred)*100:.2f}%")

Accuracy Score: 87.20%


# Test

* Geography: France
* Credit Score: 600
* Gender: Male
* Age: 40 years old
* Tenure: 3 years
* Balance: 60000
* Number of Products: 2
* Does this customer have a credit card ? Yes
* Is this customer an Active Member: Yes
* Estimated Salary: 50000
* So, should we say goodbye to that customer ?

In [17]:
# France, Germany, Spain, [Female=0/Male=1], CreditScore, Age, Tenure, Balance, NumOfProducts,
# [No=0/Yes=1], [No=0/Yes=1], EstimatedSalary

#test = [1, 0, 0, 1, 600, 40, 3, 60000, 2, 1, 1, 50000]
test = [0, 1, 0, 1, 585, 36, 5, 146050.97, 2, 0, 0, 86424.57]

print("Customer losses") if model.predict(sc.transform([test])) > 0.5 else print("Customer remains")

Customer remains
